In [8]:
import torch.utils
from torch.utils.data import DataLoader
import torchvision, torch
import numpy as np
from torchvision.transforms import v2
from torchvision import transforms
from PIL import Image
from os import sys
from torchvision import models
from torch import nn
from torch import cuda

In [9]:
def predict_image(
    model,
    image_path,
    class_names=["indoor", "outdoor"],
):
    preprocess = transforms.Compose(
        [
            transforms.Resize(560),
            transforms.CenterCrop(560 - 24),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]
    )

    image = Image.open(image_path)
    image_tensor = preprocess(image).unsqueeze(0)
    device = cuda.device("cuda") if cuda.is_available() else "cpu"

    with torch.no_grad():
        model = model.to(device)
        output = model(image_tensor)

    _, predicted = torch.max(output, 1)

    # print(f'Predicted class: {class_names[predicted.item()]}')
    print(predicted.item())
    return class_names[predicted.item()]


def predict_environment(imagepath):
    model = models.mobilenet_v3_large(weights="DEFAULT")
    num_classes = 2
    model.classifier = nn.Sequential(
        nn.Linear(in_features=960, out_features=1280, bias=True),
        nn.Hardswish(),
        nn.Dropout(p=0.2),
        nn.Linear(in_features=1280, out_features=num_classes, bias=True),
    )
    map_loc = cuda.device("cuda") if cuda.is_available() else "cpu"
    model.load_state_dict(
        torch.load("models-indoor-outdoor/best_model.pth", map_location=map_loc)
    )
    pred = predict_image(model, imagepath)
    # print(f"Predicted class: {pred}")
    return pred

In [10]:
imagepath = "./scene-images/indoor1.jpg"

pred_env = predict_environment(imagepath)
print(pred_env)

0
indoor
